# **Tiling**

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 123.6 MB/s eta 0:00:00


In [ ]:
import rasterio
import os
from google.colab import drive
import numpy as np

# Paths to the satellite imagery and binary mask
drive.mount('/content/gdrive')
satellite_image_path = "## YOUR FOLDER ##"  # Update with your file path

# Output directories for tiles
output_dir_image = "## YOUR FOLDER ##"

# Create output directories if they don't exist
os.makedirs(output_dir_image, exist_ok=True)

# Tile size
tile_size = 128

# Function to clip the raster into tiles with 50% overlap
def clip_to_tiles(input_path, output_dir, tile_size, prefix):
    with rasterio.open(input_path) as src:
        width, height = src.width, src.height
        meta = src.meta

        # Step size for 50% overlap
        step_size = tile_size // 2

        # Iterate over the raster in chunks with 50% overlap
        for i in range(0, width, step_size):
            for j in range(0, height, step_size):
                # Ensure the window does not exceed the image dimensions
                window_width = min(tile_size, width - i)
                window_height = min(tile_size, height - j)

                # Skip tiles that are smaller than the tile size
                if window_width < tile_size or window_height < tile_size:
                    continue

                # Define the window
                window = rasterio.windows.Window(i, j, window_width, window_height)
                transform = rasterio.windows.transform(window, src.transform)

                # Read the data in the window
                data = src.read(window=window)

                # Update metadata for the tile
                meta.update({
                    "height": data.shape[1],
                    "width": data.shape[2],
                    "transform": transform
                })

                # Save the tile
                tile_path = os.path.join(output_dir, f"{prefix}_image_{i}_{j}.tif")
                with rasterio.open(tile_path, "w", **meta) as dst:
                    dst.write(data)

# Clip satellite imagery
clip_to_tiles(satellite_image_path, output_dir_image, tile_size, prefix="image")

print("Clipping complete! Tiles saved in respective directories.")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Clipping complete! Tiles saved in respective directories.
